## Cuantización vectorial 

La mayoría de los embeddings vectoriales se almacenan como números de punto flotante (64 bits en Python). Podemos utilizar la cuantización para reducir el tamaño de los embeddings.


In [ ]:
# Cargamos los vectores desde movies.json, {movie: [vector]}
import json

with open('embeddings/peliculas_text-embedding-3-small-1536.json') as f:
    movies = json.load(f)

movies['La Sirenita'][0:10]

## Cuantización Escalar


In [ ]:
def scalar_quantization(embeddings: list[list[float]], num_levels=256, min_level=-128, max_level=127):
    # Aplanamos los embeddings para encontrar el mínimo y máximo global
    flattened_embeddings = [
        number for embedding in embeddings for number in embedding
    ]
    min_val = min(flattened_embeddings)
    max_val = max(flattened_embeddings)
    
    # Normalizamos los embeddings a [0, 1]
    normalized_embeddings = [
        [(number - min_val) / (max_val - min_val) for number in embedding]
        for embedding in embeddings
    ]
    
    # Cuantizamos los valores normalizados al número de niveles especificado
    quantized_embeddings = [
        [int(round(number * (num_levels - 1)) * (max_level - min_level) / (num_levels - 1) + min_level)
        for number in embedding]
        for embedding in normalized_embeddings
    ]
    
    return quantized_embeddings

quantized_embeddings = scalar_quantization(list(movies.values()))
movies_1byte = {
    movie: quantized_embedding
    for movie, quantized_embedding in zip(movies.keys(), quantized_embeddings)
}

# Comprobamos los primeros 10 bytes del vector cuantizado para 'El Rey León'
print(movies_1byte['El Rey León'][0:10])

In [ ]:
import matplotlib.pyplot as plt


def render_vector(vector):
    """Visualizar los valores del vector en un bar chart"""
    plt.bar(range(len(vector)), vector)
    plt.xlabel('Dimension')
    plt.ylabel('Value')
    plt.show()

render_vector(movies['El Rey León'])
render_vector(movies_1byte['El Rey León'])

In [ ]:
# 10 peliculas mas similares a El Rey León
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def most_similar(movie, movies):
    movie_vec = movies[movie]
    similarities = {k: cosine_similarity([movie_vec], [v])[0][0] for k, v in movies.items()}
    closest = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    df = pd.DataFrame(closest, columns=['película', 'similitud'])
    return df

most_similar('El Rey León', movies_1byte)[:10]

In [ ]:
most_similar('El Rey León', movies)[:10]

## Cuantización Binaria

La forma más extrema de cuantización consiste en almacenar los embeddings como números binarios, estableciendo cada dimensión a 0 o 1, basándose en un umbral.

In [ ]:
def binary_quantization(embeddings):
    """Convertimos un float32 en un bit basado en el umbral proporcionado"""
    
    # encontramos la media de todas las dimensiones
    flattened_embeddings = [
        number for embedding in embeddings for number in embedding
    ]
    mean_val = sum(flattened_embeddings) / len(flattened_embeddings)
    
    # cuantizamos los embeddings a 1 bit
    quantized_embeddings = [
        [1 if number > mean_val else 0 for number in embedding]
        for embedding in embeddings
    ]
    return quantized_embeddings

binary_quantized_embeddings = binary_quantization(list(movies.values()))
movies_1bit = {
    movie: quantized_embedding
    for movie, quantized_embedding in zip(movies.keys(), binary_quantized_embeddings)
}

# Comprobamos los primeros 10 bits del vector cuantizado para 'El Rey León'
print(movies_1bit['El Rey León'][0:10])

In [ ]:
render_vector(movies_1bit['El Rey León'])

In [ ]:
most_similar('El Rey León', movies_1bit)[:10]

## Comparación de tamaño

In [ ]:
import sys

pd.DataFrame({
    'float64': [sys.getsizeof(movies['El Rey León'])],
    'int8': [sys.getsizeof(movies_1byte['El Rey León'])],
    'int1': [sys.getsizeof(movies_1bit['El Rey León'])],
})

In [ ]:
# Ahora convertimos cada uno a arrays de numpy del tipo apropiado
import numpy as np
def convert_to_numpy(movies, dtype):
    return {
        movie: np.array(embedding, dtype=dtype)
        for movie, embedding in movies.items()
    }
movies_float64 = convert_to_numpy(movies, np.float64)
movies_int8 = convert_to_numpy(movies_1byte, np.int8)
movies_int1 = convert_to_numpy(movies_1bit, np.int8)

pd.DataFrame({
    'float64': [sys.getsizeof(movies_float64['El Rey León'])],
    'int8': [sys.getsizeof(movies_int8['El Rey León'])],
    'int1': [sys.getsizeof(movies_int1['El Rey León'])],
})

In [ ]:
sys.getsizeof(movies_float64['El Rey León'])/sys.getsizeof(movies_int8['Mulan'])

## Recursos

* [Scalar quantization 101](https://www.elastic.co/search-labs/blog/scalar-quantization-101)
* [Product quantization 101](https://www.pinecone.io/learn/series/faiss/product-quantization/)
* [Binary and scalar quantization](https://huggingface.co/blog/embedding-quantization)